In [1]:
from pprint import pprint
from nltk.stem.snowball import EnglishStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

from atod import Heroes

In [2]:
heroes = Heroes.all()

No abilities for this HeroID == 16


In [3]:
abilities = {h.name: list(map(lambda x: x.replace('\\n', ' ').replace('%%', ''), 
                              h.abilities.get_texts()['description']))
                     for h in heroes}

# What above dict comprehension does: 
# abilities = dict()
# for hero in heroes:
#     abilities[hero.name] = list()
#     for ability in hero.abilities.get_texts():
#         abilities[hero.name].append(ability.replace('\\n', ' ').replace('%%', '%'))
# I just love lambdas and comprehensions, so :)

In [4]:
# Create list of stop words 
# names of heroes commonly occur in descriptions, so
# there is need to remove them
heroes_names = [h.name for h in heroes]
words_in_heroes_names = [word.lower() 
                         for name in heroes_names 
                         for word in name.split(' ')]

eng_stop_words = TfidfVectorizer(stop_words='english').get_stop_words()
stop_words = set(words_in_heroes_names + list(eng_stop_words) 
                 + ['font', 'color', '7998b5', 'target', 'enemy', 'friendly', 'allied',
                    'remnant', 'aghanim', 'scepter', 'units', 'deal damage'
                    'cause', 'creep'])
stemmed_stop_words = [stemmer.stem(word) for word in stop_words]

In [5]:
stemmer = EnglishStemmer()
stemmed_corpus = dict()

# for every hero
for hero, texts in abilities.items():    
    stemmed_corpus[hero] = list()
    # concatenate all abilities descriptions into one
    # and stem all words inside
    stemmed_corpus[hero] = ' '.join([stemmer.stem(word)
                                     for doc in texts
                                     for word in doc.split(' ')])

{'Abaddon': 'abaddon releas a coil of death mist that can damag an enemi unit '
            'or heal a friend unit at the cost of some of abaddon health. '
            'summon dark energi around an alli unit, creat a shield that '
            'absorb a set amount of damag befor expiring. when the shield is '
            'destroy it will burst and deal damag equal to the amount it could '
            'absorb to an area around it. remov certain type of negat buff and '
            'stun on cast.  dispel type: strong dispel abaddon strike an enemi '
            'with chill curs on each attack, caus all unit who attack the slow '
            'enemi to gain increas movement speed, along with faster attack '
            'speed, for a limit time. when activated, all damag dealt to you '
            'will heal instead of harm. most negat buff will also be removed. '
            'if the abil is not on cooldown, it will automat activ if your '
            'health fall below %hp_threshold%.  upgr

In [9]:
cnt_vectorizer = TfidfVectorizer(stop_words=stemmed_stop_words,
                                 ngram_range=(1,2))
cnt_vectorizer.fit(stemmed_corpus.values())

(112, 6768)


In [ ]:
# find the most popular ngrams in the corpus

# try min_df tfidf